## Purpose 
 
The purpose is to assess the robustness of the ALE module implemented in TrioCFD, considering the case of a single oscillating cylinder subject to an incident laminar cross-flow. The results of our 2D numerical simulations are compared to those in [1] obtained with STAR-CCM.

 Validation made by : M. A. Puscas 
 
 Report generated  19/11/2024


In [ ]:
from trustutils import run
 
run.reset() 
run.addCase("F_0.5","ALE_Placzek.data",nbProcs=4)
run.addCase("F_0.9","ALE_Placzek.data",nbProcs=4)
run.runCases()

## Problem Description 
 
The domain used for the numerical simulations is the one in Figure 1.



### Geometry 


![](src/pb_scheme.png)

### Initial Conditions and Boundary Conditions 


We study the interaction between a moving cylinder and an incident cross-flow. The cylinder is imposed a sinusoidal displacement of frequency $F_0$ and amplitude $A$.  The frequency ratio is noted $f=F_0/F_s$ and the dimensionless amplitude is $A^*=A/D=0.25$. As in [1], our simulations are performed at a low Reynolds number $Re=DU_{\infty}/\nu=100$, with $U_{\infty}$ the incident uniform fluid velocity. For such a low Reynolds number, the flow is 2D and laminar. The computations start from the final state of the solution obtained with the fixed cylinder (from which we find a Strouhal number $St=F_s D/U_\infty= 0.167$, exactly as ref. [1]). The simulations are performed at different frequencies to cover both the lock-in and unlocked zones. A configuration is locked (resp. unlocked) if the frequency of the vortex shedding $F_s$ equals (resp. differs from) the frequency of the imposed cylinder displacement.


### Fluid Properties 


The tube bundle is immersed in a Newtonian incompressible homogeneous fluid, with mass density $\rho \,(1000 \,Kg/m^3 )$ and kinematic viscosity $\nu \,(10^{-6}\, m^2/s)$.


## Case Setup 
 


### Grid 


A refined mesh is used in the regions with large gradient fields whereas a loose mesh is used in the areas with low gradient fields. 


Calculation has been partitioned on 4 processors, such that each processor worked with 20000-30000 elements.


### Model Options 


The fluid problem with moving boundaries is handled by the Arbitrary Lagrangian-Eulerian (ALE) method.


In the ALE approach, the fluid flow is computed in a domain that is deformed in order to follow the movement of the fluid-solid interface. It provides a hybrid description not associated with the fluid particles and the laboratory coordinates. We associate the description with a moving imaginary mesh that follows the fluid domain.


## Results 
### Validation Specific Informations 
* Version TRUST : 1.9.5 
* Problem: Pb\_hydraulique\_ALE
* Dimension: 2D
* Domain: Domaine\_ALE
* Pressure solver: Solver\_moving\_mesh\_ALE PETSC Cholesky
* Discretization: VEFPre1B
* Time scheme: Scheme\_euler\_implicit with solver implicite\_ALE GMRES
* Medium: Fluide\_Incompressible
* Hydraulic problem: Navier\_Stokes\_standard\_ALE
* Convection scheme: ALE muscl
* Generated Test cases : 
	- F\_0.5/ALE\_Placzek.data :  
	- F\_0.9/ALE\_Placzek.data :  


### Performance Chart
 
 

In [ ]:
run.tablePerf()

### Plot Data 
 


#### Locked configuration


In order to highlight the lock-in phenomenous, the Power Spectral Density (PSD) is used. We can see that for $f=0.9$ there is only one peak at $f*=1$ i.e. at $f_0$ because $f*=f/f_0$ which shows that the vortex shedding is governed by the forced oscillation. The phase portraits also show that, indeed the cycle is well defined.


![](build/ReferenceSolution/ref_F0.9.png)

Locked configuration, $f=0.9$. Evolution of the lift coeﬃcient (ﬁrst column) and its PSD (second column). Reference solution [1].

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal



In [ ]:

##Import des données

F_p=np.loadtxt(run.BUILD_DIRECTORY + '/F_0.9/ALE_Placzek_pb_Force_pression.out') #Force de pression

Fv_p = np.loadtxt(run.BUILD_DIRECTORY + '/F_0.9/ALE_Placzek_pb_Contrainte_visqueuse.out') #Contrainte visqueuse

placzek = np.loadtxt(run.BUILD_DIRECTORY + '/ReferenceSolution/Placzek_Cl_alpha_0.9.dat')

##Paramètres de l'étude

Reynolds=100       #Reynolds number 
D=0.001             #cylinder diameter
nu=1.e-6            #cinematic viscosity
rho=1000            #volumic mass

V=nu*Reynolds/D     #inlet velocity
f_v = 16.87        #Strouhal frequency

## Paramètres à changer
KC=0.25            #Kalegan Karpenter number (A/D)
f0=f_v*0.9           #imposed frequency on the cylinder
nb_p_i=25        #number of period to ignore

omega=2*np.pi*f0      #pulsation
A=KC*D             #displacment amplitude
T=1/f0             #oscillation period 
ti=T*nb_p_i        #start time to compute coefficient 


# How many period are available ?
nb_p=1
for i in range(len(F_p[:,0])):
	t=F_p[i,0]
	if t > nb_p*T:
		nb_p=nb_p+1
		l=i            #the indice of the time corresponding to the period-ending is kept
tf=nb_p*T          #final time to compute coefficient 

#data selection in the time interval asked
t=0
i=0
while t < ti:
	t=F_p[i,0]
	i=i+1




##sélection des données
t= F_p[i:l,0]       #time
Fx=F_p[i:l,1]      #pressure force x direction
Fv_x=Fv_p[i:l,1]    #viscous force x direction
Fy=F_p[i:l,2]       #pressure force y direction
Fv_y=Fv_p[i:l,2]    #viscous force y direction

alpha=KC*np.sin(omega*t) #+np.full_like(t,np.pi)# #vertical displacement
v_ale=KC*omega*np.cos(omega*t) #vertical speed

Fx_t=Fx+Fv_x                #total force x direction
Fy_t=Fy+Fv_y                #total force y direction



##-------Drag------- 
Drag_mean=np.mean(Fx_t) 
Drag_std=np.std(Fx_t) 

Cd=Fx_t/(0.5*rho*V*V*D) 
Cd_mean=np.mean(Cd) 
Cd_std=np.std(Cd) 

# print(f'Cd moyen = {Cd_mean}')

 ##-------Lift------- 
Lift_mean=np.mean(Fy_t) 
Lift_std=np.std(Fy_t) 

Cl_brut=(Fy+Fv_y)/(0.5*rho*V*V*D) 

# Fenêtre de lissage (nombre de points à utiliser pour la moyenne mobile)
window = 12

# Calcul de la moyenne mobile
Cl = np.convolve(Cl_brut, np.ones(window)/window, mode='valid')

Cl_mean=np.mean(Cl) 
Cl_std=np.std(Cl) 
Cl_peak=np.max(Cl)

#print(f'Cl max = {Cl_peak}')

DataOut = np.column_stack((round(Cd_mean,3),round(Cl_peak,3)))
np.savetxt(run.BUILD_DIRECTORY + '/F_0.9/Cd_Cl.txt', DataOut)


# Color settings (CEA colors)

# Primary colours

cea_red = (229/255,0,25/255)
cea_black = (0,0,0)
cea_white = (1,1,1)
cea_darkblue = (62/255,74/255,131/255)
cea_lightblue = (126/255,156/255,187/255)
cea_darkgrey = (38/255,38/255,38/255)
cea_yellow = (1,205/255,49/255)

# Additional ones
macaron = (218/255,131/255,123/255)
archipel = (0,147/255,157/255)
glycine = (167/255,37/255,135/255)
opera = (189/255,152/255,122/255)

# Plot Settings
A = 5.5 # Want figures to be A6
plt.rc('figure', figsize=[46.82 * .5**(.5 * A), 33.11 * .5**(.5 * A)])
plt.rc('text', usetex=True)

plt.rc('font', family='serif', size=18) #font of axislabels and title, write r before label string


#Cl en fonction de t
plt.plot(t[window-1:]/T, Cl,color =cea_darkblue)
plt.xlabel(r'$t/T_0$')
plt.ylabel(r'$C_L$')
plt.ylim(-0.8,0.8)
plt.xlim(nb_p_i, nb_p)
plt.grid(True)
plt.tight_layout()
plt.title(r'Locked configuration, $f=0.9$. Evolution of the lift coeﬃcient. Current study.')
plt.savefig(run.BUILD_DIRECTORY + '/F_0.9/Cl.png', bbox_inches='tight')
plt.figure()

# Calculer la transformée de Fourier discrète (DFT) du signal
fft_result = np.fft.fft(Cl)

# Calculer la fréquence associée à chaque composante de la DFT
frequencies = np.fft.fftfreq(len(Cl), np.mean(np.diff(t)))

# Calculer la densité de puissance spectrale (PSD)
psd = np.abs(fft_result)**2 / len(Cl)


plt.plot(frequencies/f0, psd/max(psd),color = cea_darkblue)
plt.xlabel(r'$F^*=f/f_0$')
plt.xticks(np.arange(0, 5, 0.5))
plt.ylim(0,1.1)
plt.xlim(0,5)
plt.ylabel(r'$PSD_n$')
plt.grid(True)
plt.tight_layout()
plt.title(r'Locked configuration, $f=0.9$. Evolution of the normalised PSD. Current study.')
plt.savefig(run.BUILD_DIRECTORY + '/F_0.9/PSD.png', bbox_inches='tight')
plt.figure()

#Cl en fonction de alpha

plt.plot(alpha[window-1:] ,Cl, label=r'Current study', color=cea_darkblue, ls='-', zorder=1)
plt.scatter(placzek[:,0], placzek[:,1], label = r'Ref. [1], Placzek',marker='+', color = cea_red, s=30, zorder=2)

plt.xlabel(r'$\alpha$')
plt.ylabel(r'$C_L$')
#plt.ylim(-2,2)
plt.grid(True)
plt.xticks(np.arange(-0.25, 0.3, 0.1))
plt.yticks(np.arange(-0.15, 0.2, 0.05))
plt.ylim(-0.16,0.16)
plt.xlim(-0.3,0.3)
plt.legend(loc='upper center', bbox_to_anchor=(0.45, -0.15), ncol=2, fontsize = 18)
plt.tight_layout()
plt.title(r'Locked configuration, $f=0.9$. Evolution of the associated phase portrait. Current study.')
plt.savefig(run.BUILD_DIRECTORY + "/F_0.9/Cl_alpha.png", bbox_inches='tight')



#### Unlocked configuration


 In order to get out of the lock-in zone, simulations at $f=0.5$ are performed. The lift coefficient is no longer only controlled by the forced oscillations. The lift coefficient signal is no longer purely sinusoidal, this can also be seen through the phase portraits which no longer have well defined limits. We can define the cycle-to-cycle period different from the period of the signal which can now extend over several oscillations (which we will call the beating period). For $F=0.5$, the Strouhal frequency from the fixed cylinder is placed at $f_s/f_0=1/F=2$. So we can see that the cycle-to-cycle period corresponds to the strouhal period $T_s=0.5T_0$ as found by ref. [1].


![](build/ReferenceSolution/ref_F0.5.png)

Unlocked configuration, $f=0.5$. Evolution of the lift coeﬃcient (ﬁrst column) and its PSD (second column). Reference solution [1].

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal



##Import des données

F_p=np.loadtxt(run.BUILD_DIRECTORY + '/F_0.5/ALE_Placzek_pb_Force_pression.out') #Force de pression

Fv_p = np.loadtxt(run.BUILD_DIRECTORY + '/F_0.5/ALE_Placzek_pb_Contrainte_visqueuse.out') #Contrainte visqueuse

placzek = np.loadtxt(run.BUILD_DIRECTORY + '/ReferenceSolution/Placzek_Cl_alpha_0.5.dat')

##Paramètres de l'étude

Reynolds=100       #Reynolds number 
D=0.001             #cylinder diameter
nu=1.e-6            #cinematic viscosity
rho=1000            #volumic mass

V=nu*Reynolds/D     #inlet velocity
f_v = 16.87        #Strouhal frequency

## Paramètres à changer
KC=0.25            #Kalegan Karpenter number (A/D)
f0=f_v*0.5            #imposed frequency on the cylinder
nb_p_i=1        #number of period to ignore

omega=2*np.pi*f0      #pulsation
A=KC*D             #displacment amplitude
T=1/f0             #oscillation period 
ti=T*nb_p_i        #start time to compute coefficient 


# How many period are available ?
nb_p=1
for i in range(len(F_p[:,0])):
	t=F_p[i,0]
	if t > nb_p*T:
		nb_p=nb_p+1
		l=i            #the indice of the time corresponding to the period-ending is kept
tf=nb_p*T          #final time to compute coefficient 

#data selection in the time interval asked
t=0
i=0
while t < ti:
	t=F_p[i,0]
	i=i+1




##sélection des données
t= F_p[i:l,0]       #time
Fx=F_p[i:l,1]      #pressure force x direction
Fv_x=Fv_p[i:l,1]    #viscous force x direction
Fy=F_p[i:l,2]       #pressure force y direction
Fv_y=Fv_p[i:l,2]    #viscous force y direction

alpha=KC*np.sin(omega*t) #+np.full_like(t,np.pi)# #vertical displacement
v_ale=KC*omega*np.cos(omega*t) #vertical speed

Fx_t=Fx+Fv_x                #total force x direction
Fy_t=Fy+Fv_y                #total force y direction



##-------Drag------- 
Drag_mean=np.mean(Fx_t) 
Drag_std=np.std(Fx_t) 

Cd=Fx_t/(0.5*rho*V*V*D) 
Cd_mean=np.mean(Cd) 
Cd_std=np.std(Cd) 

# print(f'Cd moyen = {Cd_mean}')

 ##-------Lift------- 
Lift_mean=np.mean(Fy_t) 
Lift_std=np.std(Fy_t) 

Cl_brut=(Fy+Fv_y)/(0.5*rho*V*V*D) 

# Fenêtre de lissage (nombre de points à utiliser pour la moyenne mobile)
window = 12

# Calcul de la moyenne mobile
Cl = np.convolve(Cl_brut, np.ones(window)/window, mode='valid')

Cl_mean=np.mean(Cl) 
Cl_std=np.std(Cl) 
Cl_peak=np.max(Cl)

# print(f'Cl max = {Cl_peak}')

DataOut = np.column_stack((round(Cd_mean,3),round(Cl_peak,3)))
np.savetxt(run.BUILD_DIRECTORY + '/F_0.5/Cd_Cl.txt', DataOut)

# Color settings (CEA colors)

# Primary colours

cea_red = (229/255,0,25/255)
cea_black = (0,0,0)
cea_white = (1,1,1)
cea_darkblue = (62/255,74/255,131/255)
cea_lightblue = (126/255,156/255,187/255)
cea_darkgrey = (38/255,38/255,38/255)
cea_yellow = (1,205/255,49/255)

# Additional ones
macaron = (218/255,131/255,123/255)
archipel = (0,147/255,157/255)
glycine = (167/255,37/255,135/255)
opera = (189/255,152/255,122/255)

# Plot Settings
A = 5.5 # Want figures to be A6
plt.rc('figure', figsize=[46.82 * .5**(.5 * A), 33.11 * .5**(.5 * A)])
plt.rc('text', usetex=True)

plt.rc('font', family='serif', size=18) #font of axislabels and title, write r before label string


#Cl en fonction de t
plt.plot(t[window-1:]/T, Cl,color = cea_darkblue)
plt.xlabel(r'$t/T_0$')
plt.ylabel(r'$C_L$')
plt.ylim(-0.5,0.5)
plt.xlim(nb_p_i,9.5)
plt.grid(True)
plt.tight_layout()
plt.title(r'Unocked configuration, $f=0.5$. Evolution of the lift coeﬃcient. Current study.')
plt.savefig(run.BUILD_DIRECTORY + '/F_0.5/Cl.png', bbox_inches='tight')
plt.figure()

# Calculer la transformée de Fourier discrète (DFT) du signal
fft_result = np.fft.fft(Cl)

# Calculer la fréquence associée à chaque composante de la DFT
frequencies = np.fft.fftfreq(len(Cl), np.mean(np.diff(t)))

# Calculer la densité de puissance spectrale (PSD)
psd = np.abs(fft_result)**2 / len(Cl)


plt.plot(frequencies/f0, psd/max(psd),color = cea_darkblue)
plt.xlabel(r'$F^*=f/f_0$')
plt.xticks(np.arange(0, 5, 0.5))
plt.ylim(0,1.1)
plt.xlim(0,5)
plt.ylabel(r'$PSD_n$')


plt.grid(True)
plt.tight_layout()
plt.title(r'Unlocked configuration, $f=0.5$. Evolution of the normalised PSD. Current study.')
plt.savefig(run.BUILD_DIRECTORY + '/F_0.5/PSD.png', bbox_inches='tight')
plt.figure()

#Cl en fonction de alpha

plt.plot(alpha[window-1:] ,Cl, label=r'Current study', color=cea_darkblue, ls='-', zorder=1)
plt.scatter(placzek[:,0], placzek[:,1], label = r'Ref. [1], Placzek',marker='+', color = cea_red, s=30, zorder=2)

plt.xlabel(r'$\alpha$')
plt.ylabel(r'$C_L$')
#plt.ylim(-2,2)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.title(r'Unlocked configuration, $f=0.5$. Evolution of the associated phase portrait. Current study.')
plt.savefig(run.BUILD_DIRECTORY + "/F_0.5/Cl_alpha.png", bbox_inches='tight')


## Aerodynamic coefficients.


To study the evolution of the fluid force ${\bf{F}}$ acting on the cylinder, we introduce the drag and the lift coefficients $C_D = \frac{F_D}{1/2 \rho U_{\infty}^2 D}$ and $C_L = \frac{F_L}{1/2 \rho U_{\infty}^2 D}$,  with $F_D={\bf{F}}\cdot {\bf{e_x}}$ and $F_L={\bf{F}}\cdot {\bf{e_y}}$.


### Mean drag coefficient and the maximal lift coefficient with the frequency ratio $f = 1.1 $
 
 

In [ ]:
from trustutils import plot 
 
columns=['Mean $C_D$ ', ' Max $C_L$'] 
tab = plot.Table(columns)
tab.addLine([[1.5, 0.136]],r"Reference solution [1] ")
data = plot.loadText("F_0.9/Cd_Cl.txt", transpose=True, dtype="str")
tab.addLine([[ data[0], data[1] ]],r"Current study")
tab.setTitle("Mean drag coefficient and the maximal lift coefficient with the frequency ratio $f = 1.1 $")
display(tab)

### Mean drag coefficient and the maximal lift coefficient with the frequency ratio $f = 0.5 $
 
 

In [ ]:
from trustutils import plot 
 
columns=['Mean $C_D$ ', ' Max $C_L$'] 
tab = plot.Table(columns)
tab.addLine([[1.38, 0.418]],r"Reference solution [1] ")
data = plot.loadText("F_0.5/Cd_Cl.txt",transpose=False, dtype="str")
tab.addLine([[ data[0], data[1] ]],r"Current study")
tab.setTitle("Mean drag coefficient and the maximal lift coefficient with the frequency ratio $f = 0.5 $")
display(tab)

## Conclusion 
 
The harmonic motion of a circular wall in a quiescent viscous fluid, has been  numerically simulated. A FEV method is applied to solve such a problem in conjunction with the ALE approach. The numerical results are in line with the numerical results of [1].



## References: 
 
* 1. J. F. Placzek, A. Sigrist and A. Hamdouni. Numerical simulation of an oscillating cylinder in a cross-flow at low reynolds number: Forced and free oscillations. Computers and Fluids, 38:80–100, 2009. 
